# Organizations Activity on Twitter by party

The parameters in the cell below can be adjusted to explore other political organization and time frames.

### How to explore other  political organization?
The ***organization*** parameter can be use to aggregate organizations by their paraty. The column `subcategory` in this [this other notebook](../political_organizations.ipynb?autorun=true) show the organizations that belong each party.

***Alternatively***, you can direcly use the [organizations API](http://mediamonitoring.gesis.org/api/organizations/swagger/), or access it with the [SMM Wrapper](https://pypi.org/project/smm-wrapper/).

In [ ]:
# Parameters: 
organization <- "spd"
aggregation <- "week" # aggregation can be by day ('day'), week ('week') or month('month')
from_date <-'2017-09-01'
to_date <-'2018-12-31'

In [ ]:
library("httr")
library("jsonlite")
library("ggplot2")
library('RJSplot')
library('dplyr')

## Using API

In [ ]:
base <- "193.175.238.88/api/organizations/"
call1 <- paste(base,"all/", sep="")
get_organizations_df <- as.data.frame(fromJSON(content(GET(call1), "text", encoding="UTF-8"), flatten = TRUE))
party_df <- get_organizations_df[get_organizations_df$subcategory == organization & !is.null(get_organizations_df$tw_ids) ,]
tweets <- data.frame()
replies <- data.frame()
for (tw_id in party_df$tw_ids) {
    for (tw in tw_id) {
        call_tweets <- paste(base,"twitter/tweets_by/organizations/user_id/", tw, "/?from_date=",from_date,"&to_date=",to_date,"&aggregate_by=", aggregation, sep="")
        call_replies <- paste(base,"twitter/replies_to/organizations/user_id/", tw, "/?from_date=",from_date,"&to_date=",to_date,"&aggregate_by=", aggregation, sep="")
        if (length(fromJSON(content(GET(call_tweets), "text", encoding="UTF-8"), flatten = TRUE)$values) != 0)  {
            tweets_unique <- as.data.frame(fromJSON(content(GET(call_tweets), "text", encoding="UTF-8"), flatten = TRUE))
            tweets_unique$twitter_user_id = tw
            tweets <- rbind(tweets, tweets_unique)
            }
        if (length(fromJSON(content(GET(call_replies), "text", encoding="UTF-8"), flatten = TRUE)$values) != 0)  {
            replies_unique <- as.data.frame(fromJSON(content(GET(call_replies), "text", encoding="UTF-8"), flatten = TRUE))
            replies_unique$twitter_user_id = tw
            replies <- rbind(replies, replies_unique)
            }
    }
}

tweets <- summarise(group_by(tweets, labels), tweets = sum(values), response_type = aggregation)
replies <- summarise(group_by(replies, labels), replies = sum(values), response_type = aggregation)
merged <- merge(tweets, replies, by='labels')

In [ ]:
#plotting
options(repr.plot.width=8, repr.plot.height=4)
ggplot(data = merged, mapping = aes(as.Date(labels))) +
  geom_line(aes(y = tweets, color="Tweets", group=response_type.x)) +
  geom_line(aes(y = replies, color="Replies", group=response_type.y)) +
  labs(title = "Twitter (tweets)", x = "", y = "Tweets") +
  theme(axis.text.x = element_text( size = 7, angle = 60, hjust = 0.5, vjust = 0.5),
        axis.text.y = element_text( size = 7), legend.title = element_blank(), plot.title = element_text(size=10)) +
scale_x_date(date_breaks = "2 weeks") + scale_y_continuous(labels = scales::number_format(big.mark = ""))